In [ ]:
# First import all packages required
import numpy as np
import numpy.linalg as linalg
from scipy.optimize import linprog
from scipy.linalg import null_space
import matplotlib.pyplot as plt
import pandas as pd
import os
import escher
from escher import Builder
import cobra
import json
import yaml

# Disable warning messages for infeasible simulations
import warnings
warnings.filterwarnings('ignore')


# if more space is required
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>div.output_scroll {height: 100em; }</style>"))

# set defaults for plotting - grid on, linewidth = 2
plt.rc( 'axes', grid=True  )
plt.rc( 'figure', figsize = (7,4), dpi=96)
plt.rc( 'axes', linewidth=1 )
plt.rc( 'lines', linewidth=2 )

# change rows and columns displayed in the DataFrames 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
# load cobra model:

# Without enzyme coupling
cm = cobra.io.load_json_model('C1_hero_network_expanded_v7.json')

# With enzyme coupling
#cm = cobra.io.load_json_model('C1_hero_network_expanded_v7.json')


cm_h2o = cm.copy()

# Remove metabolites only necessary for the thermodynamic calculations
cm.remove_metabolites( cm.metabolites.get_by_id('H2O') )
cm.remove_metabolites( cm.metabolites.get_by_id('NAD') )
cm.remove_metabolites( cm.metabolites.get_by_id('NADP') )
cm.remove_metabolites( cm.metabolites.get_by_id('Pi') )


# Run if you suspect there may be blocked reactions
#cobra.flux_analysis.find_blocked_reactions(cm)


In [ ]:
# Store reversibility data for every reaction
revers_dict={}
for react in cm.reactions:
    revers_dict[react.id] = react.reversibility

In [ ]:
# %% Internal metabolic Pathways

### Sugar Pathways
    
# Embden-Meyerhof
glyc_path = cobra.core.Group("EM", "Embden-Meyerhof Glycolysis", kind="partonomy")
glyc_path.add_members([cm.reactions.PGI, cm.reactions.PFK,
                       cm.reactions.FBA1, cm.reactions.TPI,
                       cm.reactions.G3PDH, cm.reactions.PGK, cm.reactions.PGM,
                       cm.reactions.ENO, cm.reactions.PYK,
                       cm.reactions.PDC])
    
    
    
# Entner-Doundoroff pathway
ed_path = cobra.core.Group("ED", "Entner-Doundoroff pathway", kind="partonomy")
ed_path.add_members([cm.reactions.G6PDH,
                     cm.reactions.PGL6, cm.reactions.PGDH6,
                     cm.reactions.KDPG_Aldolase, cm.reactions.G3PDH,
                     cm.reactions.PGK, cm.reactions.PGM,
                     cm.reactions.ENO, cm.reactions.PYK,
                     cm.reactions.PDC])
    
# Non-oxidative glycolysis 
nog_path = cobra.core.Group("NOG", "Non-oxidative glycolysis (FPK)", kind="partonomy")
nog_path.add_members([cm.reactions.PGI,
                      cm.reactions.FPK,cm.reactions.XPK,
                      cm.reactions.TKT1, cm.reactions.TAL,
                      cm.reactions.TKT2, cm.reactions.RPI,
                      cm.reactions.RPE, cm.reactions.TPI, cm.reactions.FBA1,
                      cm.reactions.FBPase, cm.reactions.PTA])



### Methanol Pathways
    
##RuMP Cycle variants
    
# RuMP Cycle (TAL)
rump_tal_path = cobra.core.Group("RuMP Cycle (TAL)", "RumP Cycle (TAL)", kind="partonomy")
rump_tal_path.add_members([cm.reactions.MDH,
                           cm.reactions.HPS, cm.reactions.PHI,
                           cm.reactions.TKT1, cm.reactions.TKT2,
                           cm.reactions.TAL, cm.reactions.RPI,
                           cm.reactions.RPE])
    
    
    # RuMP Cycle (SBPase)
rump_sbpase_path = cobra.core.Group("RuMP Cycle (SBPase)", "RumP Cycle (SBPase)", kind="partonomy")
rump_sbpase_path.add_members([cm.reactions.MDH,
                              cm.reactions.HPS, cm.reactions.PHI,
                              cm.reactions.PFK, cm.reactions.FBA2,
                              cm.reactions.TKT1, cm.reactions.TKT2,
                              cm.reactions.SD17_Biphosphatase])
    

## Serine cycle and synthetic variants
    
# Serine cycle
ser_path = cobra.core.Group("Serine cycle", "Serine Cycle", kind="partonomy")
ser_path.add_members([cm.reactions.MDH,
                      cm.reactions.FAE, cm.reactions.MTDB,
                      cm.reactions.MCH, cm.reactions.FHC,
                      cm.reactions.FTFL, cm.reactions.FCH,
                      cm.reactions.MTDA, cm.reactions.SHMT,
                      cm.reactions.SGAT, cm.reactions.HPR,
                      cm.reactions.GK, cm.reactions.ENO,
                      cm.reactions.PPC, cm.reactions.MALDH,
                      cm.reactions.MTK, cm.reactions.MCL])
    
    
# Modified serine cycle
mod_ser_path = cobra.core.Group("Modified Serine Cycle", "Modified Serine Cycle", kind="partonomy")
mod_ser_path.add_members([cm.reactions.MDH,
                          cm.reactions.FADH, cm.reactions.FTFL,
                          cm.reactions.MTHFS, cm.reactions.SHMT,
                          cm.reactions.SDAA, cm.reactions.PPS,
                          cm.reactions.PPC, cm.reactions.MALDH,
                          cm.reactions.MTK, cm.reactions.MCL,
                          cm.reactions.AGT, cm.reactions.GPT,
                          cm.reactions.GLDH])

# Serine-Threonine Pathway
ser_thr_path = cobra.core.Group("Ser-Thr Pathway","Serine-Threonine Pathway", kind="partonomy")
ser_thr_path.add_members([cm.reactions.MDH,
                          cm.reactions.FADH, cm.reactions.FTFL,
                          cm.reactions.MTHFS, cm.reactions.SHMT,
                          cm.reactions.SDAA, cm.reactions.PPS,
                          cm.reactions.PPC, cm.reactions.ASPC,
                          cm.reactions.AK, cm.reactions.ASD,
                          cm.reactions.THRA,cm.reactions.HSK,
                          cm.reactions.TS, cm.reactions.TDH,
                          cm.reactions.KBL, cm.reactions.GLDH])
    
    
# Homoserine Pathway
homo_path = cobra.core.Group("Homoserine Pathway", "Homoserine Cycle", kind="partonomy")
homo_path.add_members([cm.reactions.MDH,
                       cm.reactions.SAL, cm.reactions.SDAA,
                       cm.reactions.HAL, cm.reactions.HAT,
                       cm.reactions.HSK, cm.reactions.TS,
                       cm.reactions.LTA, cm.reactions.ACDH,
                       cm.reactions.GLDH])
    
    
## Synthetic cycles
    
# DAS Pathway
das_path = cobra.core.Group("DAS Pathway","Dihydroxyacetone synthase pathway", kind="partonomy")
das_path.add_members([cm.reactions.MDH,
                      cm.reactions.DAS, cm.reactions.DHAK, 
                      cm.reactions.FSA, cm.reactions.TKT2])
    
    
## Synthetic linear pathways
   
# Reductive Glycine Pathway
rgly_path = cobra.core.Group("rGly Pathway", "Reductive Glycine Pathway", kind="partonomy")
rgly_path.add_members([cm.reactions.MDH,cm.reactions.FADH,
                       cm.reactions.FTFL, cm.reactions.FCH,
                       cm.reactions.MTDA, cm.reactions.GCS,
                       cm.reactions.SHMT, cm.reactions.SDAA,
                       cm.reactions.PDC])
    
    
# Formolase Pathway
for_path = cobra.core.Group("Formolase Pathway", "Formolase Pathway", kind="partonomy")
for_path.add_members([cm.reactions.MDH,cm.reactions.FADH,
                      cm.reactions.FDH,cm.reactions.ACS,
                      cm.reactions.ACDH2, cm.reactions.FLS,
                      cm.reactions.DHAK])
    
# AMP+ATP=2*ADP
    
    
# SACA Pathway
saca_path = cobra.core.Group("SACA Pathway", "Synthetic Acetyl-CoA Pathway", kind="partonomy")
saca_path.add_members([cm.reactions.MDH,
                       cm.reactions.GALS, cm.reactions.ACPS,
                       cm.reactions.PTA])
    
    
# GAA Pathway 
gaa_path = cobra.core.Group("GAA Pathway", "Glycolaldehyde Assimilation Pathway", kind="partonomy")
gaa_path.add_members([cm.reactions.MDH,
                      cm.reactions.GALS, cm.reactions.TALB_F178Y,
                      cm.reactions.RPE, cm.reactions.XPK,
                      cm.reactions.PTA])
    
    
# GAPA Pathway
gapa_path=cobra.core.Group("GAPA Pathway", "GAPA Pathway", kind="partonomy")
gapa_path.add_members([cm.reactions.MDH,
                       cm.reactions.GALS, cm.reactions.DEOC,
                       cm.reactions.RPIB, cm.reactions.ALSE])
    
# Hacl Pathway
hacl_path=cobra.core.Group("Hacl Pathway", "HacL Pathway", kind="partonomy")
hacl_path.add_members([cm.reactions.MDH,
                       cm.reactions.ACDH2, cm.reactions.HACL,
                       cm.reactions.ACR, cm.reactions.ACPS,
                       cm.reactions.PTA])
    
### Other Pathways
    
# Pentose phosphate pathway
ppp_path = cobra.core.Group("ppp", "Pentose phosphate pathway", kind="partonomy")
ppp_path.add_members([cm.reactions.G6PDH, cm.reactions.PGL6,
                     cm.reactions.PGD6, cm.reactions.RPI,
                     cm.reactions.RPE, cm.reactions.TKT1,
                     cm.reactions.TAL, cm.reactions.TKT2])
    
    
# TCA Cycle
tca_path = cobra.core.Group("tca", "TCA Cycle", kind="partonomy")
tca_path.add_members([cm.reactions.PDC, cm.reactions.CS,
                      cm.reactions.ACO, cm.reactions.IDH,
                      cm.reactions.OGDC, cm.reactions.SCS,
                      cm.reactions.SDH, cm.reactions.FUM,
                      cm.reactions.MALDH, cm.reactions.PC])

In [ ]:
# %% Product Pathways

# PHB biosynthesis
phb_path = cobra.core.Group("PHB", "PHB Synthesis", kind="partonomy")
phb_path.add_members([cm.reactions.ACAT, cm.reactions.PHBB,
                      cm.reactions.PHBC, cm.reactions.t_PHB])
    
    
# Noreugenin biosynthesis
nor_path= cobra.core.Group("Nor", "Noreugenin synthesis", kind="partonomy")
nor_path.add_members([cm.reactions.ACC, cm.reactions.PCS,
                      cm.reactions.t_Noreugenin])

# Butyric acid biosynthesis
buty_path= cobra.core.Group("Buty", "Butyric acid synthesis", kind="partonomy")
buty_path.add_members([cm.reactions.ACC, cm.reactions.FAS_BUTY,
                       cm.reactions.t_Butyric])

# Butyric acid methyl-ester biosynthesis
bame_path= cobra.core.Group("BAME", "Butyric acid methyl ester synthesis", kind="partonomy")
bame_path.add_members([cm.reactions.ACC, cm.reactions.FAS_BUTY,
                       cm.reactions.BAME_Synthesis, cm.reactions.t_BAME])

In [ ]:
# List of sugar metabolizing pathways
sugar_paths = [glyc_path, ed_path, nog_path]
sugar_nn_steps= [0, 0, 2]


# List of methanol metabolizing pathways
meth_paths = [rump_tal_path, rump_sbpase_path, ser_path, mod_ser_path, ser_thr_path, homo_path, das_path, rgly_path, for_path, saca_path, gaa_path, gapa_path, hacl_path]
meth_nn_steps = [3, 3, 11, 8, 2, 1, 2, 5, 4, 3, 3, 2, 4]

#List of product producing pathways
prod_paths= [phb_path, nor_path, capo_path, capr_path, deca_path, palm_path, pame_path, bame_path]

# List of product transport reactions
prod_react= [cm.reactions.t_PHB.id, cm.reactions.t_Noreugenin.id, cm.reactions.t_Butyric.id, cm.reactions.t_Caproic.id, cm.reactions.t_Caprylic.id, cm.reactions.t_Decanoic.id, cm.reactions.t_Palmitate.id, cm.reactions.t_PAME.id, cm.reactions.t_BAME.id]


In [ ]:
### Set objective, product pathway, substrates and FALD oxidation pathway

#Polyhydroxybutyrate
flx_obj='t_PHB'
n_c=8
obj_path= phb_path
n_e= 102
n_steps= 3

# Noreugenin
# flx_obj ='t_Noreugenin'
# n_c =10
# obj_path = nor_path
# n_e= 100
# n_steps=1

# # Butyric acid
# flx_obj= 't_Butyric'
# n_c= 4
# obj_path= buty_path
# n_e=48
# n_steps=0

# # BAME
# flx_obj='t_BAME'
# n_c=5
# obj_path=bame_path
# n_e=56
# n_steps=0

cm.objective=flx_obj

## Set substrate bounds:
cm.reactions.get_by_id('t_Glc').bounds = (0, 1)
cm.reactions.get_by_id('t_Xyl').bounds = (0, 0)
#cm.reactions.get_by_id('t_Glycerol').bounds = (0, 0)
cm.reactions.get_by_id('t_MeOH').bounds = (0, 1)

## Save the initial conditions
cm_org = cm.copy()


In [ ]:
# Load model with initial conditions
cm=cm_org.copy()

## Create dictionaries and lists to store the results 
sim_results={}
path_length=[]
carbon_eff=[]
e_eff=[]
nn_steps=[]
sim_bounds={}

# Set biomass reaction to 0
cm.reactions.get_by_id('Biomass').bounds = (0, 0)

## Set boundaries of all the pathways to combine to 0
for x in sugar_paths:
    for y in x.members:
        cm.reactions.get_by_id(y.id).bounds = (0,0)

for q in meth_paths:
    for s in q.members:
        cm.reactions.get_by_id(s.id).bounds = (0,0)

# Set boundaries of the product pathways to 0
for l in prod_paths:
    for ll in l.members:
        cm.reactions.get_by_id(ll.id).bounds = (0,0)


## Simulate with only one sugar pathway and one methanol pathway  

# Activate sugar pathway 
for m in sugar_paths:

    for n in m.members:
        if revers_dict[n.id] == True:
            cm.reactions.get_by_id(n.id).bounds= (-1000,1000)    
        else:
            cm.reactions.get_by_id(n.id).bounds= (0,1000)

    #Activate methanol pathway 
    for k in meth_paths:
        for i in k.members:
            if revers_dict[i.id] == True:
                cm.reactions.get_by_id(i.id).bounds= (-1000,1000)   
            else:
                cm.reactions.get_by_id(i.id).bounds= (0,1000)

        ## Since at the end of this "for loop" the reactions of the methanol pathway are set to 0,
        ## and some of these reactions are shared with other essential pathways, such as
        ## the sugar pathways, pentose-phosphate pathways, or FALD oxidation pathways, we need to
        ## reactivate these pathways so that the simulation can run:

        # Reactivate sugar pathway
        for n in m.members:
            if revers_dict[n.id] == True:
                cm.reactions.get_by_id(n.id).bounds= (-1000,1000)    
            else:
                cm.reactions.get_by_id(n.id).bounds= (0,1000)

        # Reactivate Pentose-Phosphate cycle
        for t in ppp_path.members:
            if revers_dict[t.id] == True:
                cm.reactions.get_by_id(t.id).bounds= (-100,100)    
            else:
                cm.reactions.get_by_id(t.id).bounds= (0,100)

        # Reactivate TCA Cycle        
        for u in tca_path.members:
            if revers_dict[u.id] == True:
                cm.reactions.get_by_id(u.id).bounds= (-1000,1000)    
            else:
                cm.reactions.get_by_id(u.id).bounds= (0,1000)

        # Reactivate FDH

        # Reactivate product pathway 
        for obj_react in obj_path.members:

            if revers_dict[obj_react.id] == True:
                cm.reactions.get_by_id(obj_react.id).bounds= (-1000,1000)    
            else:
                cm.reactions.get_by_id(obj_react.id).bounds= (0,1000)

        cm.reactions.get_by_id(flx_obj).bounds =(0,1000)

        # Add knockouts (if desired)
        cm.reactions.t_CO2.bounds=(0,0)
        cm.reactions.FADH.bounds=(-0,1000)

        #print(cobra.flux_analysis.find_blocked_reactions(cm))

        # Once all necessary pathways are activa/reactivated, proceed with the simulation
        #sol= cm.optimize()
        sol = cobra.flux_analysis.pfba(cm)
        #sol = cobra.flux_analysis.pfba(cm)

        if sol.status == 'optimal':
            
            #Store flux data
            sim_name= m.id +'+'+ k.id
            sim_results[sim_name]=np.round(sol.fluxes,6)


            #print(sim_name)
            #print(cobra.flux_analysis.find_blocked_reactions(cm))
            #print(sol.status)
            #print(cm.summary())

            # Store length of the combination (add 1 to RumP Cycle)
            total_path_length= len(m) + len(k)
            path_length.append(total_path_length)

            # Calculate and store carbon efficiency
            total_carbon_eff=(100*(n_c*cm.reactions.get_by_id(flx_obj).flux))/(6*sol.fluxes.t_Glc + 5*sol.fluxes.t_Xyl + 3*sol.fluxes.t_Glycerol + sol.fluxes.t_MeOH)
            carbon_eff.append(np.round(total_carbon_eff,2))

            # Calculate and store e-efficiency
            total_e_eff = 100*(n_e*cm.reactions.get_by_id(flx_obj).flux)/(96*sol.fluxes.t_Glc + 80*sol.fluxes.t_Xyl + 50*sol.fluxes.t_Glycerol + 18*sol.fluxes.t_MeOH)
            e_eff.append(np.round(total_e_eff,2))

            # Calculate number of non-native steps
            total_nn_steps= n_steps + sugar_nn_steps[sugar_paths.index(m)] + meth_nn_steps[meth_paths.index(k)]
            nn_steps.append(total_nn_steps)


            # Store the bounds of all the reactions
            bounds=[]
            for reaction in cm.reactions:
                bounds.append(reaction.bounds)

            sim_bounds[sim_name]=bounds  

        # Reset boundaries of used methanol pathway to 0 for the next simulation
        for i in k.members:
            cm.reactions.get_by_id(i.id).bounds = (0,0)

    # Reset boundaries of used sugar pathway to 0 for the next simulation
    for n in m.members:
        cm.reactions.get_by_id(n.id).bounds = (0,0)


        
## Create DataFrame with stored data
results_df=pd.DataFrame.from_dict(sim_results, orient='index')
ori_results_df=results_df.copy()

# Store data of the objective reaction
obj_values= results_df.loc[:,flx_obj]

# Store data for CO2 transport
co2_values= results_df.loc[:,'t_CO2']

# Remove data for all product reactions
for product in prod_react:
    results_df.pop(product)

# Insert data of objective reaction, pathway length  and carbon efficiency  
results_df.insert(0,flx_obj,obj_values)
results_df.insert(1,'Non-native steps',nn_steps)
results_df.insert(2,'CO2',co2_values)
results_df.insert(3,'C-eff (%)',carbon_eff)
results_df.insert(4,'e-eff (%)', e_eff)

# Sort the data by highest objective flux
sorted_results_df = results_df.sort_values(by=flx_obj, ascending=False)


# Store the rates for every simulation and reactions in an Excel document. Change filename for the one desired for the user.
with pd.ExcelWriter("Filename.xlsx", mode="w") as writer:
    sorted_results_df.to_excel(writer, sheet_name='Summary', encoding='UTF-8')
sorted_results_df
#ori_results_df

# Save bounds for troubleshooting
react_names=[]
for re in cm.reactions:
    react_names.append(re.id)
bounds_df=pd.DataFrame.from_dict(sim_bounds, orient='index', columns=react_names)
bounds_df.to_excel("sim_bounds.xlsx")


# Display dataframe with simulations results
sorted_results_df



In [ ]:
# Sort original values from the simulations
ori_sorted_results_df = ori_results_df.sort_values(by=flx_obj, ascending=False)

# Save results into a list
ori_sorted_results_list=ori_sorted_results_df.values.tolist()

# Create  list including the reactions with H2O, the flux of each reaction , the IDs and the name of the reaction
for row_indx, simulation in enumerate(ori_sorted_results_df.index):

    # Only saves the combinations for which there is product production, use of sugars AND methanol, and no CO2 emissions
    if (float(sorted_results_df.loc[[simulation],flx_obj])) > 0 and float(sorted_results_df.loc[[simulation],'C-eff (%)']) > 99:
    #float(sorted_results_df.loc[[simulation],'t_MeOH']) > 0: #and (float(sorted_results_df.loc[[simulation],'t_Glc']) > 0 or float(sorted_results_df.loc[[simulation],'t_Xyl']) > 0): 
    
        l_h2o=[]
        for react_index,react_h2o in enumerate(cm.reactions):
            r_h2o = cm_h2o.reactions.get_by_id(react_h2o.id)
            l_h2o.append( (r_h2o.reaction, round( ori_sorted_results_list[row_indx][react_index], 6) , 0, r_h2o.id, r_h2o.name, '', '' ) )



        h2o_df = pd.DataFrame( l_h2o, columns = ['Reaction Formula','Relative Flux','Absolute Flux(M/s)','Reaction Name','Enzyme','kinetic parameters', 'Standard dG (kJ/mol)'] )
        h2o_df["Reaction Formula"] = h2o_df["Reaction Formula"].apply( lambda x: x.replace("-->", "<=>"))

        #Remove reactions with 0 flux
        h2o_df_act_flx = h2o_df[abs(h2o_df['Relative Flux']) > 1e-3]
        #print(h2o_df_act_flx)

        # remove extracellular fluxes:
        flx_incl = [ x[0:2] != 't_' for x in h2o_df_act_flx['Reaction Name'] ]
        #print(flx_incl)

        h2o_df_act_flx = h2o_df_act_flx[ flx_incl ]
        #df_act_flx = df
        #display( h2o_df_act_flx )

        filename= str(row_indx+1) +'__'+ simulation + '__to_equil.csv'
        filename= filename.replace('+','_')

        # Save in Excel file for visualization
        sim_sheet= str(row_indx+1) +'__'+ simulation
        
        with pd.ExcelWriter("PHB_Glc&MeOH_EnzCoup_FBA.xlsx", mode="a", engine="openpyxl") as writer:
            h2o_df_act_flx.to_excel(writer, sheet_name=sim_sheet, encoding='UTF-8')
    
    else:
        continue